## 调度问题举例

In [1]:
from ortools.sat.python import cp_model

In [17]:
# 创建模型
model = cp_model.CpModel()

In [18]:
# 定义数据
jobs_data = [
    # (处理时间, 截止时间)
    (3, 7),  # 任务 0
    (2, 4),  # 任务 1
    (4, 10)  # 任务 2
]
num_jobs = len(jobs_data)

In [19]:
# 定义变量
starts = []
ends = []
for i in range(num_jobs):
    duration = jobs_data[i][0]
    latest_end = jobs_data[i][1]
    start_var = model.NewIntVar(0, latest_end, f'start_{i}')
    end_var = model.NewIntVar(0, latest_end, f'end_{i}')
    starts.append(start_var)
    ends.append(end_var)
    model.Add(start_var + duration == end_var)

In [20]:
# 确保任务不重叠
model.AddNoOverlap([model.NewIntervalVar(starts[i], jobs_data[i][0], ends[i], f'interval_{i}')
                    for i in range(num_jobs)])

In [21]:
# 定义一个新变量表示最大结束时间
max_end = model.NewIntVar(0, max(job[1] for job in jobs_data), 'max_end')
for end_var in ends:
    model.Add(max_end >= end_var)

In [22]:
# 定义目标函数：最小化最大结束时间
model.Minimize(max_end)

In [10]:
# 创建求解器并求解
solver = cp_model.CpSolver()
status = solver.Solve(model)

# 打印解决方案
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total length of the schedule: {solver.ObjectiveValue()}')
    for i in range(num_jobs):
        print(f'Job {i} starts at {solver.Value(starts[i])} and ends at {solver.Value(ends[i])}')
else:
    print("No solution found.")

Total length of the schedule: 9.0
Job 0 starts at 2 and ends at 5
Job 1 starts at 0 and ends at 2
Job 2 starts at 5 and ends at 9


## 回调函数打印所有的~~最优~~（可行）解

- https://developers.google.com/optimization/cp/cp_solver?hl=zh-cn

In [12]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.Value(v)}", end=" ")
        print()

    def solution_count(self):
        return self.__solution_count

In [23]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([*starts, *ends])
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

In [24]:
# Solve.
status = solver.Solve(model, solution_printer)

start_0=2 start_1=0 start_2=5 end_0=5 end_1=2 end_2=9 


In [28]:
status 

4

In [29]:
cp_model.OPTIMAL

4

In [27]:
solver.Value(starts[1])

0